# Forth protein mapping implementation

Main idea:
- get from CATH domain_ids + cath_domain_coordinates
- get from RSCB protein_seq
- search in RSCB for  rscb_domain_coordinates
  - exact match of domain_id
  - cath_domain_coordinates
  - length
  - seq alignment
- cut domain_seq from protein using rscb_domain_coordinates

Problem:
- still some of domain_ids could not be mapped

TODO:
- сheck seq alignment
- ~~automate remaining exceptions~~ make research if it even makes sense

In [9]:
import pandas as pd

df = pd.read_csv("../data/subset.csv")
required_columns = ["domain_parts", "length", "cath_domain_start1", "cath_domain_end1"]
total_rows = len(df)

incomplete_rows_df = df[df[required_columns].isnull().any(axis=1)]
incomplete_rows = len(incomplete_rows_df)

print(f"{incomplete_rows} rows with missing required fields out of {total_rows} total rows")

domain_column = 'domain_id'
incomplete_ids = set(incomplete_rows_df[domain_column].dropna().tolist())

fasta_path = "../data/cath-domain-seqs.fa"
results = {}

with open(fasta_path, "r") as file:
    for line in file:
        if line.startswith(">"):
            header = line.strip()
            last_part = header.split('|')[-1]
            if '/' in last_part:
                domain_id, region = last_part.split('/')
                if domain_id in incomplete_ids:
                    results[domain_id] = region


for domain_id, region in results.items():
    print(f"{domain_id}: {region}")

0 rows with missing required fields out of 11774 total rows


New problem:
- more than 1k domains have multiple sites
- now they all are parsed

TODO:
- make some research
- find algorithm to work with them

In [3]:
import os
import glob
import pandas as pd
import requests
from tqdm import tqdm
from Bio import SeqIO
from Bio.Align import PairwiseAligner

checkpoint_dir = "../data/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_files = glob.glob(os.path.join(checkpoint_dir, "mapped_checkpoint_*.csv"))
if checkpoint_files:
    latest_checkpoint = max(checkpoint_files, key=os.path.getctime)
    df = pd.read_csv(latest_checkpoint)
    print(f"[Resume] Loaded checkpoint: {latest_checkpoint}")
else:
    df = pd.read_csv("../data/subset.csv")
    df["protein_sequence"] = ""
    max_parts = df["domain_parts"].max()
    for i in range(1, max_parts + 1):
        df[f"domain_start{i}"] = None
        df[f"domain_end{i}"] = None
        df[f"domain_sequence{i}"] = ""
    print("[Start] No checkpoint found. Starting from scratch.")

unprocessed_mask = df["protein_sequence"].isnull() | (df["protein_sequence"].str.strip() == "")
unprocessed_indices = df[unprocessed_mask].index

cath_fasta = {
    record.id.split("|")[-1].split("/")[0]: record
    for record in SeqIO.parse("../data/cath-domain-seqs.fa", "fasta")
}

pdb_overrides = {
    "1vx4K02": {"pdb": "3j7p", "chain": "K"},
    "4hubI02": {"pdb": "4v9f", "chain": "I"},
    "7cosB02": {"pdb": "8igw", "chain": "B"},
    "1mslA02": {"pdb": "2oar", "chain": "A"},
    "3s6n201": {"pdb": "5xjl", "chain": "2"},
    "2j01501": {"pdb": "4v51", "chain": "B5"},
    "3u5cE04": {"pdb": "4v88", "chain": "CE"},
    "2v49501": {"pdb": "4v5a", "chain": "B5"},
    "3rjrB02": {"pdb": "5vqf", "chain": "B"},
    "3o58g00": {"pdb": "4v7r", "chain": "Bg"},
    "3o58d00": {"pdb": "4v7r", "chain": "Dg"},
    "1vwxA02": {"pdb": "3j7o", "chain": "A"},
    "4i4m600": {"pdb": "4v9f", "chain": "6"},
    "4a18X01": {"pdb": "4v8p", "chain": "AX"},
    "4p6vF01": {"pdb": "8acy", "chain": "F"},
    "2m25A00": {"pdb": "2n17", "chain": "A"},
    "1vx4407": {"pdb": "3j7p", "chain": "4"},
    "3kitJ00": {"pdb": "4v7j", "chain": "AJ"},
    "1vx7200": {"pdb": "3j79", "chain": "2"},
    "1vwxr00": {"pdb": "3j7o", "chain": "r"},
    "4a18O00": {"pdb": "4v8p", "chain": "AO"},
    "1vx2P00": {"pdb": "3j7r", "chain": "SP"},
    "3o30I00": {"pdb": "4v7r", "chain": "AI"},
    "3u5eJ00": {"pdb": "4v88", "chain": "BJ"},
    "4k0mC02": {"pdb": "4v9i", "chain": "BC"},
    "3o58A02": {"pdb": "4v7r", "chain": "BA"},
    "7yyqA01": {"pdb": "8ql2", "chain": "A"},
    "3o58D00": {"pdb": "4v7r", "chain": "Dd"},
    "1vw3D00": {"pdb": "3j6b", "chain": "D"},
    "1vwxP00": {"pdb": "3j7o", "chain": "P"},
    "1vx7W00": {"pdb": "3j79", "chain": "W"},
    "3o58J00": {"pdb": "4v7r", "chain": "BJ"},
    "3kypE01": {"pdb": "5x7v", "chain": "E"},
    "1vw4801": {"pdb": "3j6b", "chain": "8"},
    "1vw4F02": {"pdb": "3j6b", "chain": "F"},
    "1vw4F01": {"pdb": "3j6b", "chain": "F"},
    "4gnsA01": {"pdb": "4yg8", "chain": "A"},
    "1vs9S01": {"pdb": "4v4i", "chain": "S"},
    "3p9dA03": {"pdb": "4v81", "chain": "A"},
    "4d8qF03": {"pdb": "4v94", "chain": "F"},
    "4a17E01": {"pdb": "4v8p", "chain": "BE"},
}

def get_fasta_sequence_with_label(pdb_id, auth_chain_id):
    url = f"https://www.rcsb.org/fasta/entry/{pdb_id}"
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return "", None

        fasta_blocks = response.text.strip().split(">")
        for block in fasta_blocks:
            lines = block.strip().splitlines()
            if not lines:
                continue
            header = lines[0]
            sequence = "".join(lines[1:])

            if "|Chain " in header or "|Chains " in header:
                chain_field = header.split("|")[1]
                parts = chain_field.replace("Chains ", "").replace("Chain ", "").split(",")
                for part in parts:
                    part = part.strip()
                    if "[auth " in part:
                        model_id = part.split("[auth")[0].strip()
                        auth_id = part.split("[auth")[1].replace("]", "").strip()
                        if auth_chain_id == auth_id:
                            return sequence, model_id
                    else:
                        if auth_chain_id == part:
                            return sequence, auth_chain_id
        return "", None
    except Exception as e:
        print(f"[Error] Failed to fetch FASTA for PDB ID {pdb_id}, chain {auth_chain_id}: {e}")
        return "", None

def get_cath_coordinates(pdb_id, chain_id, domain_id, expected_length, domain_start=None, domain_end=None, reference_seq=None, protein_seq=None):
    url = f"https://data.rcsb.org/rest/v1/core/polymer_entity_instance/{pdb_id}/{chain_id}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()

        features = data.get("rcsb_polymer_instance_feature", [])
        matched_coords = []

        # Step 1: match by domain_id
        for feature in features:
            if feature.get("type") != "CATH":
                continue
            for prop in feature.get("additional_properties", []):
                if prop.get("name") == "CATH_DOMAIN_ID":
                    ids = [v.strip().lower() for v in prop.get("values", [])]
                    if domain_id.strip().lower() in ids:
                        coords = feature.get("feature_positions", [])
                        matched_coords.extend([(c["beg_seq_id"], c["end_seq_id"]) for c in coords])

        if matched_coords:
            return matched_coords

        # Step 2: match by domain start/end
        if domain_start and domain_end:
            for feature in features:
                if feature.get("type") != "CATH":
                    continue
                coords = feature.get("feature_positions", [])
                for c in coords:
                    if abs(c["beg_seq_id"] - domain_start) <= 2 and abs(c["end_seq_id"] - domain_end) <= 2:
                        matched_coords.append((c["beg_seq_id"], c["end_seq_id"]))
            if matched_coords:
                return matched_coords

        # Step 3: match by domain length
        for feature in features:
            if feature.get("type") != "CATH":
                continue
            coords = feature.get("feature_positions", [])
            for c in coords:
                length = c["end_seq_id"] - c["beg_seq_id"] + 1
                if expected_length - 2 <= length <= expected_length + 2:
                    matched_coords.append((c["beg_seq_id"], c["end_seq_id"]))
        if matched_coords:
            return matched_coords

        # Step 4: global sequence alignment
        if reference_seq and protein_seq:
            aligner = PairwiseAligner()
            best_score = -1
            best_start = None
            for i in range(0, len(protein_seq) - expected_length + 1):
                window = protein_seq[i:i+expected_length]
                score = aligner.score(reference_seq[:expected_length], window)
                if score > best_score:
                    best_score = score
                    best_start = i
            if best_start is not None:
                return [(best_start + 1, best_start + expected_length)]

        print(f"[No Match] Could not map {domain_id}")
        return []

    except Exception as e:
        print(f"[Error] Failed to fetch CATH data for {pdb_id} {chain_id}: {e}")
        return []

aligner = PairwiseAligner()

for idx in tqdm(unprocessed_indices, total=len(unprocessed_indices)):
    row = df.loc[idx]

    domain_id = row.get("domain_id", "")
    if not isinstance(domain_id, str) or len(domain_id) < 6:
        continue

    if domain_id in pdb_overrides:
        pdb_id = pdb_overrides[domain_id]["pdb"]
        auth_chain_id = pdb_overrides[domain_id]["chain"]
    else:
        pdb_id = domain_id[:4].lower()
        auth_chain_id = domain_id[4]

    ref_seq_record = cath_fasta.get(domain_id)
    reference_seq = str(ref_seq_record.seq) if ref_seq_record else None

    protein_seq, label_asym_id = get_fasta_sequence_with_label(pdb_id, auth_chain_id)
    if not protein_seq or not label_asym_id:
        continue

    df.at[idx, "protein_sequence"] = protein_seq

    n_parts = int(row.get("domain_parts", 1))
    segment_lengths = []

    for i in range(1, n_parts + 1):
        start = row.get(f"cath_domain_start{i}")
        end = row.get(f"cath_domain_end{i}")
        if pd.notna(start) and pd.notna(end):
            start, end = int(start), int(end)
            segment_lengths.append(end - start + 1)

    expected_length = sum(segment_lengths)

    initial_coords = get_cath_coordinates(
        pdb_id, label_asym_id, domain_id, expected_length,
        domain_start=None, domain_end=None,
        reference_seq=reference_seq, protein_seq=protein_seq
    )

    coordinates = initial_coords

    if not coordinates:
        coordinates = []
        search_start = 0
        for seg_len in segment_lengths:
            best_score = -1
            best_start = None
            for j in range(search_start, len(protein_seq) - seg_len + 1):
                window = protein_seq[j:j + seg_len]
                score = aligner.score(reference_seq[:seg_len], window)
                if score > best_score:
                    best_score = score
                    best_start = j
            if best_start is None:
                coordinates = []
                break
            coordinates.append((best_start + 1, best_start + seg_len))
            search_start = best_start + seg_len

    if not coordinates or len(coordinates) != n_parts:
        print(f"[Skip] {domain_id} → could not match all parts.")
        continue

    full_seq = ""
    for i, (start, end) in enumerate(coordinates, 1):
        part_seq = protein_seq[start - 1:end]
        df.at[idx, f"domain_start{i}"] = start
        df.at[idx, f"domain_end{i}"] = end
        df.at[idx, f"domain_sequence{i}"] = part_seq
        full_seq += part_seq

    df.at[idx, "domain_sequence1"] = full_seq

    if (idx + 1) % 10 == 0:
        df.to_csv("../data/subset_protein_mapped.csv", index=False)
        checkpoint_path = os.path.join(checkpoint_dir, f"mapped_checkpoint_{idx + 1}.csv")
        df.to_csv(checkpoint_path, index=False)
        print(f"[Checkpoint] Saved progress at row {idx + 1}")

df.to_csv("../data/subset_protein_mapped.csv", index=False)
print("[Final] Saved full dataframe")

[Resume] Loaded checkpoint: ../data/checkpoints/mapped_checkpoint_10190.csv


  2%|▏         | 26/1600 [00:16<28:31,  1.09s/it]

[Checkpoint] Saved progress at row 10200


  2%|▏         | 36/1600 [00:26<29:15,  1.12s/it]

[Checkpoint] Saved progress at row 10210


...

[Checkpoint] Saved progress at row 11760


100%|█████████▉| 1596/1600 [26:12<00:04,  1.14s/it]

[Checkpoint] Saved progress at row 11770


100%|██████████| 1600/1600 [26:16<00:00,  1.02it/s]


[Final] Saved full dataframe


In [5]:
import pandas as pd

df = pd.read_csv("../data/subset_protein_mapped.csv")

incomplete_mask = df["protein_sequence"].isnull() | df["domain_sequence1"].isnull()
incomplete_rows = df[incomplete_mask]
problem_ids = incomplete_rows["domain_id"].tolist()
print("Domain IDs with missing protein_sequence or domain_sequence1:")
for d in problem_ids:
    print(d)

print(f"\nTotal incomplete rows: {len(problem_ids)} out of {len(df)} total rows.")

#df = df[~incomplete_mask]
#df.to_csv("../data/subset_protein_mapped.csv", index=False)

Domain IDs with missing protein_sequence or domain_sequence1:
4u2vA02
3hm6X01
1werA02
1nf1A01
3ziuB01
6spoA03
3fnrA01
3ziuA01
1gaxA02
3bnjA01
1oahA01
3t6pA01
1vw4900
1vw4501
2a01C01
1khdA01
4eadA01
5noeA01
2qtlA01
1f20A02
2bpoA02
4kbqD00
7zo9A01
7bi4A01
8amzQ01
1v1hB00
6e56D01
1n8zC04
2b39A06
2d1sA01
1rfeA01
3b5mA01
2ausA01
1rcqA01
5x7nB01
2nvaA01
2vd8A01
2yxxA01
5gjoB01
4eclA01
4beuA01
1knwA01
1d7kB01
5zl6A01
5fagA01
3n29B01
4aibA01
2pljA01
4lusB01
2j66A01
7odcA01
3n2bD01
3n2oC01
1pfzA02
5bnzA04
2hz7A04
4en2A02
1u0tA02
4k35A02
4pvaA00
3pijA01
5hp6A01
3ugfA01
8fm5A01
1vzrA01
3au4A02
2grvC03
6l4lA01
4k8lA01
4dunA01
2gkeA01
4jbdA01
2otnB01
6hjfB01
3ednA01
5k87A01
2gkeA02
6lpnA01
5yuyA01
3u27A01
4hubG01
1vx4q01
7uwrA02
6l3mB01
7s6dB02
1vw4502
4wweA01
3o58A01
3lltA01
2fstX01
7si1A01
6ks6B02
6ks6e02
8bm1K01
7d5mA01
3btvA02
3db2B02
2ixaA02
7xr9E01
3dtyB02
1lc0A02
1vw4700
1l0wA02
1v4pC01
1tkeA02
2ztgA04
7qh7701
4bt2A01
1xv2C01
4pabA03
2gagC01
1pj5A03
1v5vA01
3girA01
4dx5A04
7whfC01
1ti2A02
3u

To be fixed:

* 4u2vA02
* 3hm6X01
* 1werA02
* 1nf1A01
* 3ziuB01
* 6spoA03
* 3fnrA01
* 3ziuA01
* 1gaxA02
* 3bnjA01
* 1oahA01
* 3t6pA01
* 1vw4900
* 1vw4501
* 2a01C01
* 1khdA01
* 4eadA01
* 5noeA01
* 2qtlA01
* 1f20A02
* 2bpoA02
* 4kbqD00
* 7zo9A01
* 7bi4A01
* 8amzQ01
* 1v1hB00
* 6e56D01
* 1n8zC04
* 2b39A06
* 2d1sA01
* 1rfeA01
* 3b5mA01
* 2ausA01
* 1rcqA01
* 5x7nB01
* 2nvaA01
* 2vd8A01
* 2yxxA01
* 5gjoB01
* 4eclA01
* 4beuA01
* 1knwA01
* 1d7kB01
* 5zl6A01
* 5fagA01
* 3n29B01
* 4aibA01
* 2pljA01
* 4lusB01
* 2j66A01
* 7odcA01
* 3n2bD01
* 3n2oC01
* 1pfzA02
* 5bnzA04
* 2hz7A04
* 4en2A02
* 1u0tA02
* 4k35A02
* 4pvaA00
* 3pijA01
* 5hp6A01
* 3ugfA01
* 8fm5A01
* 1vzrA01
* 3au4A02
* 2grvC03
* 6l4lA01
* 4k8lA01
* 4dunA01
* 2gkeA01
* 4jbdA01
* 2otnB01
* 6hjfB01
* 3ednA01
* 5k87A01
* 2gkeA02
* 6lpnA01
* 5yuyA01
* 3u27A01
* 4hubG01
* 1vx4q01
* 7uwrA02
* 6l3mB01
* 7s6dB02
* 1vw4502
* 4wweA01
* 3o58A01
* 3lltA01
* 2fstX01
* 7si1A01
* 6ks6B02
* 6ks6e02
* 8bm1K01
* 7d5mA01
* 3btvA02
* 3db2B02
* 2ixaA02
* 7xr9E01
* 3dtyB02
* 1lc0A02
* 1vw4700
* 1l0wA02
* 1v4pC01
* 1tkeA02
* 2ztgA04
* 7qh7701
* 4bt2A01
* 1xv2C01
* 4pabA03
* 2gagC01
* 1pj5A03
* 1v5vA01
* 3girA01
* 4dx5A04
* 7whfC01
* 1ti2A02
* 3uykA01
* 2qzsA01
* 8inpA01
* 2iuyA01
* 4x7rA01
* 1f0kA01
* 3ia7A01
* 3t7dA01
* 2iw1A01
* 3s5jA02
* 1u0tB01
* 3pfnD01
* 1z0sA01
* 1y3iA01
* 3s40A01
* 1yt5A01
* 2i2cA01
* 2bonA01
* 3afoB01
* 1u0tA01
* 7r4lA01
* 2an1D01
* 4werA01
* 6cngA01
* 3jr7A01
* 1pzxB01
* 1oi2A01
* 3pl5A01
* 6spcb01
* 2e5lB01
* 7o0aD01
* 2amlB01
* 4bb9A02
* 7ntgA01
* 2yimA01
* 1x74A01
* 1q7eA01
* 1xk7A01
* 4ed9A01
* 8edjA01
* 1ti2A03
* 5kf6B02
* 2uvaJ10
* 6fkuA01
* 4c3sA01
* 3v4cB01
* 6jqlA01
* 7w5kA01
* 3r64A01
* 3pqaB01
* 3ju8A01
* 1o04A01
* 8dr9A01
* 4jbeB01
* 4o6rA01
* 3szaA01
* 4ogdA01
* 3k2wD01
* 5ekcF01
* 5qr1A01
* 5yuyA02
* 3o58D00
* 4kctA01
* 2x1dA01
* 5u81A01
* 2aafB01
* 1s9rA01
* 7kz9A01
* 7kz9B01
* 2o7iA02
* 3ubtA02
* 4dv8A01
* 6qdiA02
* 6qdiA01
* 3dorA03
* 3bf0C03
* 1uyvB02
* 1vw4102
* 2ckfC01
* 2bmoA01
* 2de6A01
* 3vcaA01
* 3gzxA01
* 2b1xA01
* 1z01A01
* 3k8kA03
* 4aeeA04
* 1g5aA03
* 7phoB01
* 3afhA02
* 1vs9F02
* 1vw4K00
* 1vw4H00
* 3o58Y02
* 4a17N00